In [51]:
# Add execute permissions to binaries
!chmod +x dssp
!chmod +x DAlphaBall.gcc

# Verify permissions
!ls -l dssp
!ls -l DAlphaBall.gcc

-rwxr-xr-x@ 1 satishgaurav  staff  877904 May  4 21:24 dssp
-rwxr-xr-x@ 1 satishgaurav  staff  345824 May  4 21:24 DAlphaBall.gcc


In [52]:
from IPython.display import display, Markdown

In [53]:
####################################
################ BioPython functions
####################################
### Import dependencies
import os
import math
import numpy as np
from collections import defaultdict
from scipy.spatial import cKDTree
from Bio import BiopythonWarning
from Bio.PDB import PDBParser, DSSP, Selection, Polypeptide, PDBIO, Select, Chain, Superimposer
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from Bio.PDB.Selection import unfold_entities
from Bio.PDB.Polypeptide import is_aa

# analyze sequence composition of design
def validate_design_sequence(sequence, num_clashes, advanced_settings):
    note_array = []

    # Check if protein contains clashes after relaxation
    if num_clashes > 0:
        note_array.append('Relaxed structure contains clashes.')

    # Check if the sequence contains disallowed amino acids
    if advanced_settings["omit_AAs"]:
        restricted_AAs = advanced_settings["omit_AAs"].split(',')
        for restricted_AA in restricted_AAs:
            if restricted_AA in sequence:
                note_array.append('Contains: '+restricted_AA+'!')

    # Analyze the protein
    analysis = ProteinAnalysis(sequence)

    # Calculate the reduced extinction coefficient per 1% solution
    extinction_coefficient_reduced = analysis.molar_extinction_coefficient()[0]
    molecular_weight = round(analysis.molecular_weight() / 1000, 2)
    extinction_coefficient_reduced_1 = round(extinction_coefficient_reduced / molecular_weight * 0.01, 2)

    # Check if the absorption is high enough
    if extinction_coefficient_reduced_1 <= 2:
        note_array.append(f'Absorption value is {extinction_coefficient_reduced_1}, consider adding tryptophane to design.')

    # Join the notes into a single string
    notes = ' '.join(note_array)

    return notes

# temporary function, calculate RMSD of input PDB and trajectory target
def target_pdb_rmsd(trajectory_pdb, starting_pdb, chain_ids_string):
    # Parse the PDB files
    parser = PDBParser(QUIET=True)
    structure_trajectory = parser.get_structure('trajectory', trajectory_pdb)
    structure_starting = parser.get_structure('starting', starting_pdb)
    
    # Extract chain A from trajectory_pdb
    chain_trajectory = structure_trajectory[0]['A']
    
    # Extract the specified chains from starting_pdb
    chain_ids = chain_ids_string.split(',')
    residues_starting = []
    for chain_id in chain_ids:
        chain_id = chain_id.strip()
        chain = structure_starting[0][chain_id]
        for residue in chain:
            if is_aa(residue, standard=True):
                residues_starting.append(residue)
    
    # Extract residues from chain A in trajectory_pdb
    residues_trajectory = [residue for residue in chain_trajectory if is_aa(residue, standard=True)]
    
    # Ensure that both structures have the same number of residues
    min_length = min(len(residues_starting), len(residues_trajectory))
    residues_starting = residues_starting[:min_length]
    residues_trajectory = residues_trajectory[:min_length]
    
    # Collect CA atoms from the two sets of residues
    atoms_starting = [residue['CA'] for residue in residues_starting if 'CA' in residue]
    atoms_trajectory = [residue['CA'] for residue in residues_trajectory if 'CA' in residue]
    
    # Calculate RMSD using structural alignment
    sup = Superimposer()
    sup.set_atoms(atoms_starting, atoms_trajectory)
    rmsd = sup.rms
    
    return round(rmsd, 2)

# detect C alpha clashes for deformed trajectories
def calculate_clash_score(pdb_file, threshold=2.4, only_ca=False):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('protein', pdb_file)

    atoms = []
    atom_info = []  # Detailed atom info for debugging and processing

    for model in structure:
        for chain in model:
            for residue in chain:
                for atom in residue:
                    if atom.element == 'H':  # Skip hydrogen atoms
                        continue
                    if only_ca and atom.get_name() != 'CA':
                        continue
                    atoms.append(atom.coord)
                    atom_info.append((chain.id, residue.id[1], atom.get_name(), atom.coord))

    tree = cKDTree(atoms)
    pairs = tree.query_pairs(threshold)

    valid_pairs = set()
    for (i, j) in pairs:
        chain_i, res_i, name_i, coord_i = atom_info[i]
        chain_j, res_j, name_j, coord_j = atom_info[j]

        # Exclude clashes within the same residue
        if chain_i == chain_j and res_i == res_j:
            continue

        # Exclude directly sequential residues in the same chain for all atoms
        if chain_i == chain_j and abs(res_i - res_j) == 1:
            continue

        # If calculating sidechain clashes, only consider clashes between different chains
        if not only_ca and chain_i == chain_j:
            continue

        valid_pairs.add((i, j))

    return len(valid_pairs)

three_to_one_map = {
    'ALA': 'A', 'CYS': 'C', 'ASP': 'D', 'GLU': 'E', 'PHE': 'F',
    'GLY': 'G', 'HIS': 'H', 'ILE': 'I', 'LYS': 'K', 'LEU': 'L',
    'MET': 'M', 'ASN': 'N', 'PRO': 'P', 'GLN': 'Q', 'ARG': 'R',
    'SER': 'S', 'THR': 'T', 'VAL': 'V', 'TRP': 'W', 'TYR': 'Y'
}

# identify interacting residues at the binder interface
def hotspot_residues(trajectory_pdb, binder_chain="B", atom_distance_cutoff=4.0):
    # Parse the PDB file
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("complex", trajectory_pdb)

    # Get the specified chain
    binder_atoms = Selection.unfold_entities(structure[0][binder_chain], 'A')
    binder_coords = np.array([atom.coord for atom in binder_atoms])

    # Get atoms and coords for the target chain
    target_atoms = Selection.unfold_entities(structure[0]['A'], 'A')
    target_coords = np.array([atom.coord for atom in target_atoms])

    # Build KD trees for both chains
    binder_tree = cKDTree(binder_coords)
    target_tree = cKDTree(target_coords)

    # Prepare to collect interacting residues
    interacting_residues = {}

    # Query the tree for pairs of atoms within the distance cutoff
    pairs = binder_tree.query_ball_tree(target_tree, atom_distance_cutoff)

    # Process each binder atom's interactions
    for binder_idx, close_indices in enumerate(pairs):
        binder_residue = binder_atoms[binder_idx].get_parent()
        binder_resname = binder_residue.get_resname()

        # Convert three-letter code to single-letter code using the manual dictionary
        if binder_resname in three_to_one_map:
            aa_single_letter = three_to_one_map[binder_resname]
            for close_idx in close_indices:
                target_residue = target_atoms[close_idx].get_parent()
                interacting_residues[binder_residue.id[1]] = aa_single_letter

    return interacting_residues

# calculate secondary structure percentage of design
def calc_ss_percentage(pdb_file, advanced_settings, chain_id="B", atom_distance_cutoff=4.0):
    # Parse the structure
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('protein', pdb_file)
    model = structure[0]  # Consider only the first model in the structure

    # Calculate DSSP for the model
    dssp = DSSP(model, pdb_file, dssp='mkdssp')

    # Prepare to count residues
    ss_counts = defaultdict(int)
    ss_interface_counts = defaultdict(int)
    plddts_interface = []
    plddts_ss = []

    # Get chain and interacting residues once
    chain = model[chain_id]
    interacting_residues = set(hotspot_residues(pdb_file, chain_id, atom_distance_cutoff).keys())

    for residue in chain:
        residue_id = residue.id[1]
        if (chain_id, residue_id) in dssp:
            ss = dssp[(chain_id, residue_id)][2]  # Get the secondary structure
            ss_type = 'loop'
            if ss in ['H', 'G', 'I']:
                ss_type = 'helix'
            elif ss == 'E':
                ss_type = 'sheet'

            ss_counts[ss_type] += 1

            if ss_type != 'loop':
                # calculate secondary structure normalised pLDDT
                avg_plddt_ss = sum(atom.bfactor for atom in residue) / len(residue)
                plddts_ss.append(avg_plddt_ss)

            if residue_id in interacting_residues:
                ss_interface_counts[ss_type] += 1

                # calculate interface pLDDT
                avg_plddt_residue = sum(atom.bfactor for atom in residue) / len(residue)
                plddts_interface.append(avg_plddt_residue)

    # Calculate percentages
    total_residues = sum(ss_counts.values())
    total_interface_residues = sum(ss_interface_counts.values())

    percentages = calculate_percentages(total_residues, ss_counts['helix'], ss_counts['sheet'])
    interface_percentages = calculate_percentages(total_interface_residues, ss_interface_counts['helix'], ss_interface_counts['sheet'])

    i_plddt = round(sum(plddts_interface) / len(plddts_interface) / 100, 2) if plddts_interface else 0
    ss_plddt = round(sum(plddts_ss) / len(plddts_ss) / 100, 2) if plddts_ss else 0

    return (*percentages, *interface_percentages, i_plddt, ss_plddt)

def calculate_percentages(total, helix, sheet):
    helix_percentage = round((helix / total) * 100,2) if total > 0 else 0
    sheet_percentage = round((sheet / total) * 100,2) if total > 0 else 0
    loop_percentage = round(((total - helix - sheet) / total) * 100,2) if total > 0 else 0

    return helix_percentage, sheet_percentage, loop_percentage

In [54]:
advanced_settings = {
    "dssp_path": "/usr/local/bin/mkdssp",
    "omit_AAs": "C",
}
pdb_file = "./../target/1yi5.pdb"
calc_ss_percentage(pdb_file=pdb_file, advanced_settings=advanced_settings)

(8.33, 44.12, 47.55, 15.79, 52.63, 31.58, 0.39, 0.39)

In [55]:
import pandas as pd

AA_PROPERTIES = {
        "ALA": "nonpolar",
        "ARG": "positive",
        "ASN": "polar",
        "ASP": "negative",
        "CYS": "polar",
        "GLN": "polar",
        "GLU": "negative",
        "GLY": "nonpolar",
        "HIS": "positive",
        "ILE": "nonpolar",
        "LEU": "nonpolar",
        "LYS": "positive",
        "MET": "nonpolar",
        "PHE": "nonpolar",
        "PRO": "nonpolar",
        "SER": "polar",
        "THR": "polar",
        "TRP": "nonpolar",
        "TYR": "polar",
        "VAL": "nonpolar",
        "SEC": "polar"
    }

def create_structure_df(pdb_file, advanced_settings, chain_id=None, atom_distance_cutoff=4.0):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('protein', pdb_file)
    model = structure[0]
    
    # Calculate DSSP for the model
    dssp = DSSP(model, pdb_file, dssp='mkdssp')
    
    # Prepare data for DataFrame
    data = []
    
    # Process either specific chain or all chains
    chains = [model[chain_id]] if chain_id else model.get_chains()
    
    for chain in chains:
        current_chain_id = chain.id
        # Get interacting residues if chain_id is specified
        interacting_residues = set()
        if current_chain_id:
            interacting_residues = set(hotspot_residues(pdb_file, current_chain_id, atom_distance_cutoff).keys())
        
        for residue in chain:
            residue_id = residue.id[1]
            res_name = residue.get_resname()
            
            if (current_chain_id, residue_id) in dssp:
                dssp_data = dssp[(current_chain_id, residue_id)]
                ss = dssp_data[2]
                acc = dssp_data[3]
                phi = dssp_data[4]
                psi = dssp_data[5]
                
                ss_type = 'loop'
                if ss in ['H', 'G', 'I']:
                    ss_type = 'helix'
                elif ss == 'E':
                    ss_type = 'sheet'
                    
                avg_plddt = sum(atom.bfactor for atom in residue) / len(residue) / 100
                
                data.append({
                    'chain_id': current_chain_id,
                    'residue_id': residue_id,
                    'residue_name': res_name,
                    'property': AA_PROPERTIES.get(res_name, 'unknown'),
                    'ss_type': ss_type,
                    'ss_code': ss,
                    'accessibility': acc,
                    'phi': phi,
                    'psi': psi,
                    'plddt': round(avg_plddt, 2),
                    'is_interface': residue_id in interacting_residues if chain_id else None
                })
    
    return pd.DataFrame(data)


# Example usage:
pdb_file = "./../target/1yi5.pdb"
df = create_structure_df(pdb_file, advanced_settings)
df.head()

,chain_id,residue_id,residue_name,property,ss_type,ss_code,accessibility,phi,psi,plddt,is_interface
0,A,1,LEU,nonpolar,loop,-,0.719512,360.0,127.0,0.39,None
1,A,2,ASP,negative,loop,-,0.282209,-129.9,161.8,0.39,None
2,A,3,ARG,positive,helix,H,0.084677,-37.6,-52.7,0.39,None
3,A,4,ALA,nonpolar,helix,H,0.132075,-54.6,-52.6,0.39,None
4,A,5,ASP,negative,helix,H,0.441718,-63.2,-22.2,0.39,None


In [56]:
df.tail(10)

,chain_id,residue_id,residue_name,property,ss_type,ss_code,accessibility,phi,psi,plddt,is_interface
1346,J,59,THR,polar,loop,S,0.669014,-85.5,132.6,0.39,None
1347,J,60,ASP,negative,loop,S,0.680982,-46.9,108.6,0.39,None
1348,J,61,ASN,polar,loop,S,0.337580,60.3,80.0,0.39,None
1349,J,62,CYS,polar,loop,-,0.207407,-121.2,1.8,0.39,None
1350,J,63,ASN,polar,loop,-,0.019108,-127.0,30.2,0.39,None
1351,J,64,PRO,nonpolar,loop,-,0.470588,-64.0,-176.1,0.39,None
1352,J,65,PHE,nonpolar,loop,-,0.375635,-69.0,132.3,0.39,None
1353,J,66,PRO,nonpolar,loop,-,0.419118,-85.9,-179.3,0.40,None
1354,J,67,THR,polar,loop,-,0.894366,-139.4,76.7,0.41,None
1355,J,68,ARG,positive,loop,-,0.576613,-158.9,360.0,0.42,None


In [57]:
df.head(10)

,chain_id,residue_id,residue_name,property,ss_type,ss_code,accessibility,phi,psi,plddt,is_interface
0,A,1,LEU,nonpolar,loop,-,0.719512,360.0,127.0,0.39,None
1,A,2,ASP,negative,loop,-,0.282209,-129.9,161.8,0.39,None
2,A,3,ARG,positive,helix,H,0.084677,-37.6,-52.7,0.39,None
3,A,4,ALA,nonpolar,helix,H,0.132075,-54.6,-52.6,0.39,None
4,A,5,ASP,negative,helix,H,0.441718,-63.2,-22.2,0.39,None
5,A,6,ILE,nonpolar,helix,H,0.011834,-76.3,-44.9,0.39,None
6,A,7,LEU,nonpolar,helix,H,0.085366,-65.9,-27.1,0.39,None
7,A,8,TYR,polar,helix,H,0.563063,-67.9,-40.0,0.40,None
8,A,9,ASN,polar,helix,H,0.515924,-55.9,-62.9,0.39,None
9,A,10,ILE,nonpolar,helix,H,0.017751,-64.9,-50.8,0.39,None


In [58]:
df_target = df[df['chain_id'] == 'F']
df_target = df_target.reset_index(drop=True)
df_target.index = range(1, len(df_target) + 1)

In [59]:
df_target.to_csv('1yi5_chain_F_secondary_structure_information.csv', index=False)

In [60]:
df_target[df_target['property'] == 'nonpolar']

,chain_id,residue_id,residue_name,property,ss_type,ss_code,accessibility,phi,psi,plddt,is_interface
1,F,1,ILE,nonpolar,loop,-,0.165680,360.0,135.5,0.39,None
4,F,4,PHE,nonpolar,sheet,E,0.258883,-69.4,112.2,0.39,None
5,F,5,ILE,nonpolar,loop,-,0.319527,-95.9,150.5,0.39,None
7,F,7,PRO,nonpolar,loop,S,0.294118,-93.0,23.6,0.39,None
9,F,9,ILE,nonpolar,loop,S,0.650888,-79.6,-14.2,0.40,None
15,F,15,PRO,nonpolar,loop,S,0.794118,-59.1,117.1,0.40,None
17,F,17,GLY,nonpolar,loop,-,0.142857,-83.0,112.5,0.40,None
19,F,19,VAL,nonpolar,sheet,E,0.126761,-137.0,141.5,0.39,None
25,F,25,TRP,nonpolar,sheet,E,0.149780,-147.4,149.1,0.39,None
28,F,28,ALA,nonpolar,loop,S,0.028302,-85.5,-40.6,0.39,None


In [61]:
display(Markdown(df_target[df_target['property'] == 'nonpolar'].sort_values('accessibility', ascending=False).to_markdown()))

|    | chain_id   |   residue_id | residue_name   | property   | ss_type   | ss_code   |   accessibility |    phi |    psi |   plddt | is_interface   |
|---:|:-----------|-------------:|:---------------|:-----------|:----------|:----------|----------------:|-------:|-------:|--------:|:---------------|
| 15 | F          |           15 | PRO            | nonpolar   | loop      | S         |       0.794118  |  -59.1 |  117.1 |    0.4  |                |
|  9 | F          |            9 | ILE            | nonpolar   | loop      | S         |       0.650888  |  -79.6 |  -14.2 |    0.4  |                |
| 48 | F          |           48 | VAL            | nonpolar   | loop      | -         |       0.619718  | -112.8 |  115.6 |    0.4  |                |
| 51 | F          |           51 | GLY            | nonpolar   | loop      | S         |       0.619048  |   98.2 |  -37   |    0.39 |                |
| 43 | F          |           43 | ALA            | nonpolar   | loop      | S         |       0.603774  |  -63.4 |  -49.8 |    0.39 |                |
| 64 | F          |           64 | PRO            | nonpolar   | loop      | -         |       0.492647  |  -64.6 | -177.9 |    0.39 |                |
| 66 | F          |           66 | PRO            | nonpolar   | loop      | -         |       0.433824  |  -78   |  176.3 |    0.4  |                |
| 65 | F          |           65 | PHE            | nonpolar   | loop      | -         |       0.365482  |  -73   |  133.8 |    0.4  |                |
| 46 | F          |           46 | PRO            | nonpolar   | loop      | P         |       0.330882  |  -74.9 |  168.4 |    0.39 |                |
|  5 | F          |            5 | ILE            | nonpolar   | loop      | -         |       0.319527  |  -95.9 |  150.5 |    0.39 |                |
|  7 | F          |            7 | PRO            | nonpolar   | loop      | S         |       0.294118  |  -93   |   23.6 |    0.39 |                |
| 54 | F          |           54 | ILE            | nonpolar   | loop      | -         |       0.284024  | -118   |  161.8 |    0.4  |                |
|  4 | F          |            4 | PHE            | nonpolar   | sheet     | E         |       0.258883  |  -69.4 |  112.2 |    0.39 |                |
| 42 | F          |           42 | ALA            | nonpolar   | sheet     | E         |       0.254717  | -163.5 |  168.6 |    0.39 |                |
| 32 | F          |           32 | ILE            | nonpolar   | helix     | H         |       0.224852  |  -84.1 |  -73   |    0.39 |                |
| 34 | F          |           34 | GLY            | nonpolar   | loop      | -         |       0.214286  |  114.6 | -177.9 |    0.39 |                |
| 52 | F          |           52 | VAL            | nonpolar   | loop      | -         |       0.197183  |  -51.9 |  143.8 |    0.39 |                |
|  1 | F          |            1 | ILE            | nonpolar   | loop      | -         |       0.16568   |  360   |  135.5 |    0.39 |                |
| 25 | F          |           25 | TRP            | nonpolar   | sheet     | E         |       0.14978   | -147.4 |  149.1 |    0.39 |                |
| 17 | F          |           17 | GLY            | nonpolar   | loop      | -         |       0.142857  |  -83   |  112.5 |    0.4  |                |
| 19 | F          |           19 | VAL            | nonpolar   | sheet     | E         |       0.126761  | -137   |  141.5 |    0.39 |                |
| 39 | F          |           39 | LEU            | nonpolar   | sheet     | E         |       0.054878  |  -90   |  166.5 |    0.39 |                |
| 29 | F          |           29 | PHE            | nonpolar   | loop      | T         |       0.0507614 |  -69.6 |   23.4 |    0.39 |                |
| 40 | F          |           40 | GLY            | nonpolar   | sheet     | E         |       0.047619  |  162.2 | -171.9 |    0.39 |                |
| 28 | F          |           28 | ALA            | nonpolar   | loop      | S         |       0.0283019 |  -85.5 |  -40.6 |    0.39 |                |
| 37 | F          |           37 | VAL            | nonpolar   | sheet     | E         |       0.0211268 | -134.8 |  121.2 |    0.39 |                |

In [62]:
display(Markdown(df_target.sort_values('accessibility', ascending=False).to_markdown()))

|    | chain_id   |   residue_id | residue_name   | property   | ss_type   | ss_code   |   accessibility |    phi |    psi |   plddt | is_interface   |
|---:|:-----------|-------------:|:---------------|:-----------|:----------|:----------|----------------:|-------:|-------:|--------:|:---------------|
| 50 | F          |           50 | THR            | polar      | loop      | S         |       1         |  -75   |  122.2 |    0.4  |                |
| 47 | F          |           47 | THR            | polar      | loop      | P         |       0.978873  |  -69   |  154.8 |    0.39 |                |
| 67 | F          |           67 | THR            | polar      | loop      | -         |       0.901408  | -107.5 |  -22.5 |    0.4  |                |
| 15 | F          |           15 | PRO            | nonpolar   | loop      | S         |       0.794118  |  -59.1 |  117.1 |    0.4  |                |
| 49 | F          |           49 | LYS            | positive   | loop      | -         |       0.756098  |  -72.8 |  153.3 |    0.4  |                |
| 44 | F          |           44 | THR            | polar      | loop      | S         |       0.753521  | -138.3 |  169.9 |    0.4  |                |
| 55 | F          |           55 | GLN            | polar      | loop      | -         |       0.717172  | -157.3 |  103.9 |    0.39 |                |
| 58 | F          |           58 | SER            | polar      | loop      | -         |       0.715385  | -141.9 |   12.9 |    0.39 |                |
| 59 | F          |           59 | THR            | polar      | loop      | S         |       0.683099  |  -88.4 |  133.8 |    0.39 |                |
| 16 | F          |           16 | ASN            | polar      | loop      | S         |       0.675159  |    8.7 |   70.7 |    0.39 |                |
| 60 | F          |           60 | ASP            | negative   | loop      | S         |       0.662577  |  -50   |  107.9 |    0.4  |                |
| 13 | F          |           13 | ASP            | negative   | sheet     | E         |       0.656442  |  -90.2 |  102.3 |    0.4  |                |
|  9 | F          |            9 | ILE            | nonpolar   | loop      | S         |       0.650888  |  -79.6 |  -14.2 |    0.4  |                |
| 18 | F          |           18 | HIS            | positive   | loop      | -         |       0.63587   |  -70.6 |   -3.3 |    0.4  |                |
| 48 | F          |           48 | VAL            | nonpolar   | loop      | -         |       0.619718  | -112.8 |  115.6 |    0.4  |                |
| 51 | F          |           51 | GLY            | nonpolar   | loop      | S         |       0.619048  |   98.2 |  -37   |    0.39 |                |
| 43 | F          |           43 | ALA            | nonpolar   | loop      | S         |       0.603774  |  -63.4 |  -49.8 |    0.39 |                |
| 53 | F          |           53 | ASP            | negative   | loop      | B         |       0.521472  | -123.2 |  126.6 |    0.4  |                |
| 68 | F          |           68 | ARG            | positive   | loop      | -         |       0.516129  | -178.2 |  360   |    0.4  |                |
| 45 | F          |           45 | CYS            | polar      | loop      | P         |       0.503704  |  -65.3 |  135.3 |    0.39 |                |
| 64 | F          |           64 | PRO            | nonpolar   | loop      | -         |       0.492647  |  -64.6 | -177.9 |    0.39 |                |
|  8 | F          |            8 | ASP            | negative   | loop      | S         |       0.466258  |  -89.2 |  152.2 |    0.4  |                |
| 12 | F          |           12 | LYS            | positive   | sheet     | E         |       0.458537  | -140.3 |  161.5 |    0.4  |                |
| 10 | F          |           10 | THR            | polar      | loop      | S         |       0.43662   | -101.8 |  165.2 |    0.39 |                |
| 66 | F          |           66 | PRO            | nonpolar   | loop      | -         |       0.433824  |  -78   |  176.3 |    0.4  |                |
| 31 | F          |           31 | SER            | polar      | helix     | H         |       0.415385  |  -65.1 |  -25.5 |    0.39 |                |
|  2 | F          |            2 | ARG            | positive   | sheet     | E         |       0.41129   |  -77.2 |  117.5 |    0.39 |                |
| 65 | F          |           65 | PHE            | nonpolar   | loop      | -         |       0.365482  |  -73   |  133.8 |    0.4  |                |
| 61 | F          |           61 | ASN            | polar      | loop      | S         |       0.33121   |   63.7 |   81   |    0.39 |                |
| 46 | F          |           46 | PRO            | nonpolar   | loop      | P         |       0.330882  |  -74.9 |  168.4 |    0.39 |                |
| 35 | F          |           35 | LYS            | positive   | loop      | -         |       0.326829  |  -58.7 |  146.8 |    0.39 |                |
| 56 | F          |           56 | CYS            | polar      | sheet     | E         |       0.325926  |  -93.7 |  162.2 |    0.39 |                |
| 23 | F          |           23 | LYS            | positive   | sheet     | E         |       0.321951  |  -69.2 |  138.9 |    0.4  |                |
|  5 | F          |            5 | ILE            | nonpolar   | loop      | -         |       0.319527  |  -95.9 |  150.5 |    0.39 |                |
| 38 | F          |           38 | ASP            | negative   | sheet     | E         |       0.319018  | -106   |  114.1 |    0.39 |                |
| 26 | F          |           26 | CYS            | polar      | loop      | -         |       0.318519  |  -93   |  119.6 |    0.4  |                |
| 36 | F          |           36 | ARG            | positive   | sheet     | E         |       0.314516  |  -73.8 |  146.9 |    0.39 |                |
|  7 | F          |            7 | PRO            | nonpolar   | loop      | S         |       0.294118  |  -93   |   23.6 |    0.39 |                |
| 11 | F          |           11 | SER            | polar      | sheet     | E         |       0.292308  |  -94   |  143.7 |    0.39 |                |
| 54 | F          |           54 | ILE            | nonpolar   | loop      | -         |       0.284024  | -118   |  161.8 |    0.4  |                |
| 62 | F          |           62 | CYS            | polar      | loop      | -         |       0.281481  | -125.6 |    6.3 |    0.39 |                |
|  4 | F          |            4 | PHE            | nonpolar   | sheet     | E         |       0.258883  |  -69.4 |  112.2 |    0.39 |                |
| 42 | F          |           42 | ALA            | nonpolar   | sheet     | E         |       0.254717  | -163.5 |  168.6 |    0.39 |                |
| 57 | F          |           57 | CYS            | polar      | sheet     | E         |       0.244444  | -173.1 |  177.3 |    0.39 |                |
| 41 | F          |           41 | CYS            | polar      | sheet     | E         |       0.237037  | -122.4 |  162.8 |    0.4  |                |
| 32 | F          |           32 | ILE            | nonpolar   | helix     | H         |       0.224852  |  -84.1 |  -73   |    0.39 |                |
| 34 | F          |           34 | GLY            | nonpolar   | loop      | -         |       0.214286  |  114.6 | -177.9 |    0.39 |                |
| 52 | F          |           52 | VAL            | nonpolar   | loop      | -         |       0.197183  |  -51.9 |  143.8 |    0.39 |                |
| 21 | F          |           21 | TYR            | polar      | sheet     | E         |       0.193694  | -124.1 |  153.5 |    0.39 |                |
| 24 | F          |           24 | THR            | polar      | sheet     | E         |       0.190141  | -149.2 |  124.5 |    0.4  |                |
|  1 | F          |            1 | ILE            | nonpolar   | loop      | -         |       0.16568   |  360   |  135.5 |    0.39 |                |
| 25 | F          |           25 | TRP            | nonpolar   | sheet     | E         |       0.14978   | -147.4 |  149.1 |    0.39 |                |
| 17 | F          |           17 | GLY            | nonpolar   | loop      | -         |       0.142857  |  -83   |  112.5 |    0.4  |                |
| 22 | F          |           22 | THR            | polar      | sheet     | E         |       0.140845  | -135.3 |   84.4 |    0.39 |                |
|  6 | F          |            6 | THR            | polar      | loop      | S         |       0.140845  | -112.8 |  147.8 |    0.39 |                |
| 19 | F          |           19 | VAL            | nonpolar   | sheet     | E         |       0.126761  | -137   |  141.5 |    0.39 |                |
| 14 | F          |           14 | CYS            | polar      | loop      | -         |       0.118519  | -106.6 |  100   |    0.4  |                |
| 27 | F          |           27 | ASP            | negative   | loop      | -         |       0.0858896 | -101.9 | -149.4 |    0.39 |                |
| 39 | F          |           39 | LEU            | nonpolar   | sheet     | E         |       0.054878  |  -90   |  166.5 |    0.39 |                |
| 29 | F          |           29 | PHE            | nonpolar   | loop      | T         |       0.0507614 |  -69.6 |   23.4 |    0.39 |                |
| 33 | F          |           33 | ARG            | positive   | helix     | H         |       0.0483871 |  -95.1 |   -2.2 |    0.39 |                |
| 40 | F          |           40 | GLY            | nonpolar   | sheet     | E         |       0.047619  |  162.2 | -171.9 |    0.39 |                |
| 30 | F          |           30 | CYS            | polar      | helix     | H         |       0.0444444 | -107.2 |  -36.9 |    0.39 |                |
| 63 | F          |           63 | ASN            | polar      | loop      | -         |       0.0382166 | -126.8 |   31   |    0.39 |                |
| 28 | F          |           28 | ALA            | nonpolar   | loop      | S         |       0.0283019 |  -85.5 |  -40.6 |    0.39 |                |
| 37 | F          |           37 | VAL            | nonpolar   | sheet     | E         |       0.0211268 | -134.8 |  121.2 |    0.39 |                |
| 20 | F          |           20 | CYS            | polar      | sheet     | E         |       0         |  -95.5 |  130.2 |    0.39 |                |
|  3 | F          |            3 | CYS            | polar      | sheet     | E         |       0         | -126.1 |  150.2 |    0.39 |                |

In [63]:
display(Markdown(df_target.to_markdown()))

|    | chain_id   |   residue_id | residue_name   | property   | ss_type   | ss_code   |   accessibility |    phi |    psi |   plddt | is_interface   |
|---:|:-----------|-------------:|:---------------|:-----------|:----------|:----------|----------------:|-------:|-------:|--------:|:---------------|
|  1 | F          |            1 | ILE            | nonpolar   | loop      | -         |       0.16568   |  360   |  135.5 |    0.39 |                |
|  2 | F          |            2 | ARG            | positive   | sheet     | E         |       0.41129   |  -77.2 |  117.5 |    0.39 |                |
|  3 | F          |            3 | CYS            | polar      | sheet     | E         |       0         | -126.1 |  150.2 |    0.39 |                |
|  4 | F          |            4 | PHE            | nonpolar   | sheet     | E         |       0.258883  |  -69.4 |  112.2 |    0.39 |                |
|  5 | F          |            5 | ILE            | nonpolar   | loop      | -         |       0.319527  |  -95.9 |  150.5 |    0.39 |                |
|  6 | F          |            6 | THR            | polar      | loop      | S         |       0.140845  | -112.8 |  147.8 |    0.39 |                |
|  7 | F          |            7 | PRO            | nonpolar   | loop      | S         |       0.294118  |  -93   |   23.6 |    0.39 |                |
|  8 | F          |            8 | ASP            | negative   | loop      | S         |       0.466258  |  -89.2 |  152.2 |    0.4  |                |
|  9 | F          |            9 | ILE            | nonpolar   | loop      | S         |       0.650888  |  -79.6 |  -14.2 |    0.4  |                |
| 10 | F          |           10 | THR            | polar      | loop      | S         |       0.43662   | -101.8 |  165.2 |    0.39 |                |
| 11 | F          |           11 | SER            | polar      | sheet     | E         |       0.292308  |  -94   |  143.7 |    0.39 |                |
| 12 | F          |           12 | LYS            | positive   | sheet     | E         |       0.458537  | -140.3 |  161.5 |    0.4  |                |
| 13 | F          |           13 | ASP            | negative   | sheet     | E         |       0.656442  |  -90.2 |  102.3 |    0.4  |                |
| 14 | F          |           14 | CYS            | polar      | loop      | -         |       0.118519  | -106.6 |  100   |    0.4  |                |
| 15 | F          |           15 | PRO            | nonpolar   | loop      | S         |       0.794118  |  -59.1 |  117.1 |    0.4  |                |
| 16 | F          |           16 | ASN            | polar      | loop      | S         |       0.675159  |    8.7 |   70.7 |    0.39 |                |
| 17 | F          |           17 | GLY            | nonpolar   | loop      | -         |       0.142857  |  -83   |  112.5 |    0.4  |                |
| 18 | F          |           18 | HIS            | positive   | loop      | -         |       0.63587   |  -70.6 |   -3.3 |    0.4  |                |
| 19 | F          |           19 | VAL            | nonpolar   | sheet     | E         |       0.126761  | -137   |  141.5 |    0.39 |                |
| 20 | F          |           20 | CYS            | polar      | sheet     | E         |       0         |  -95.5 |  130.2 |    0.39 |                |
| 21 | F          |           21 | TYR            | polar      | sheet     | E         |       0.193694  | -124.1 |  153.5 |    0.39 |                |
| 22 | F          |           22 | THR            | polar      | sheet     | E         |       0.140845  | -135.3 |   84.4 |    0.39 |                |
| 23 | F          |           23 | LYS            | positive   | sheet     | E         |       0.321951  |  -69.2 |  138.9 |    0.4  |                |
| 24 | F          |           24 | THR            | polar      | sheet     | E         |       0.190141  | -149.2 |  124.5 |    0.4  |                |
| 25 | F          |           25 | TRP            | nonpolar   | sheet     | E         |       0.14978   | -147.4 |  149.1 |    0.39 |                |
| 26 | F          |           26 | CYS            | polar      | loop      | -         |       0.318519  |  -93   |  119.6 |    0.4  |                |
| 27 | F          |           27 | ASP            | negative   | loop      | -         |       0.0858896 | -101.9 | -149.4 |    0.39 |                |
| 28 | F          |           28 | ALA            | nonpolar   | loop      | S         |       0.0283019 |  -85.5 |  -40.6 |    0.39 |                |
| 29 | F          |           29 | PHE            | nonpolar   | loop      | T         |       0.0507614 |  -69.6 |   23.4 |    0.39 |                |
| 30 | F          |           30 | CYS            | polar      | helix     | H         |       0.0444444 | -107.2 |  -36.9 |    0.39 |                |
| 31 | F          |           31 | SER            | polar      | helix     | H         |       0.415385  |  -65.1 |  -25.5 |    0.39 |                |
| 32 | F          |           32 | ILE            | nonpolar   | helix     | H         |       0.224852  |  -84.1 |  -73   |    0.39 |                |
| 33 | F          |           33 | ARG            | positive   | helix     | H         |       0.0483871 |  -95.1 |   -2.2 |    0.39 |                |
| 34 | F          |           34 | GLY            | nonpolar   | loop      | -         |       0.214286  |  114.6 | -177.9 |    0.39 |                |
| 35 | F          |           35 | LYS            | positive   | loop      | -         |       0.326829  |  -58.7 |  146.8 |    0.39 |                |
| 36 | F          |           36 | ARG            | positive   | sheet     | E         |       0.314516  |  -73.8 |  146.9 |    0.39 |                |
| 37 | F          |           37 | VAL            | nonpolar   | sheet     | E         |       0.0211268 | -134.8 |  121.2 |    0.39 |                |
| 38 | F          |           38 | ASP            | negative   | sheet     | E         |       0.319018  | -106   |  114.1 |    0.39 |                |
| 39 | F          |           39 | LEU            | nonpolar   | sheet     | E         |       0.054878  |  -90   |  166.5 |    0.39 |                |
| 40 | F          |           40 | GLY            | nonpolar   | sheet     | E         |       0.047619  |  162.2 | -171.9 |    0.39 |                |
| 41 | F          |           41 | CYS            | polar      | sheet     | E         |       0.237037  | -122.4 |  162.8 |    0.4  |                |
| 42 | F          |           42 | ALA            | nonpolar   | sheet     | E         |       0.254717  | -163.5 |  168.6 |    0.39 |                |
| 43 | F          |           43 | ALA            | nonpolar   | loop      | S         |       0.603774  |  -63.4 |  -49.8 |    0.39 |                |
| 44 | F          |           44 | THR            | polar      | loop      | S         |       0.753521  | -138.3 |  169.9 |    0.4  |                |
| 45 | F          |           45 | CYS            | polar      | loop      | P         |       0.503704  |  -65.3 |  135.3 |    0.39 |                |
| 46 | F          |           46 | PRO            | nonpolar   | loop      | P         |       0.330882  |  -74.9 |  168.4 |    0.39 |                |
| 47 | F          |           47 | THR            | polar      | loop      | P         |       0.978873  |  -69   |  154.8 |    0.39 |                |
| 48 | F          |           48 | VAL            | nonpolar   | loop      | -         |       0.619718  | -112.8 |  115.6 |    0.4  |                |
| 49 | F          |           49 | LYS            | positive   | loop      | -         |       0.756098  |  -72.8 |  153.3 |    0.4  |                |
| 50 | F          |           50 | THR            | polar      | loop      | S         |       1         |  -75   |  122.2 |    0.4  |                |
| 51 | F          |           51 | GLY            | nonpolar   | loop      | S         |       0.619048  |   98.2 |  -37   |    0.39 |                |
| 52 | F          |           52 | VAL            | nonpolar   | loop      | -         |       0.197183  |  -51.9 |  143.8 |    0.39 |                |
| 53 | F          |           53 | ASP            | negative   | loop      | B         |       0.521472  | -123.2 |  126.6 |    0.4  |                |
| 54 | F          |           54 | ILE            | nonpolar   | loop      | -         |       0.284024  | -118   |  161.8 |    0.4  |                |
| 55 | F          |           55 | GLN            | polar      | loop      | -         |       0.717172  | -157.3 |  103.9 |    0.39 |                |
| 56 | F          |           56 | CYS            | polar      | sheet     | E         |       0.325926  |  -93.7 |  162.2 |    0.39 |                |
| 57 | F          |           57 | CYS            | polar      | sheet     | E         |       0.244444  | -173.1 |  177.3 |    0.39 |                |
| 58 | F          |           58 | SER            | polar      | loop      | -         |       0.715385  | -141.9 |   12.9 |    0.39 |                |
| 59 | F          |           59 | THR            | polar      | loop      | S         |       0.683099  |  -88.4 |  133.8 |    0.39 |                |
| 60 | F          |           60 | ASP            | negative   | loop      | S         |       0.662577  |  -50   |  107.9 |    0.4  |                |
| 61 | F          |           61 | ASN            | polar      | loop      | S         |       0.33121   |   63.7 |   81   |    0.39 |                |
| 62 | F          |           62 | CYS            | polar      | loop      | -         |       0.281481  | -125.6 |    6.3 |    0.39 |                |
| 63 | F          |           63 | ASN            | polar      | loop      | -         |       0.0382166 | -126.8 |   31   |    0.39 |                |
| 64 | F          |           64 | PRO            | nonpolar   | loop      | -         |       0.492647  |  -64.6 | -177.9 |    0.39 |                |
| 65 | F          |           65 | PHE            | nonpolar   | loop      | -         |       0.365482  |  -73   |  133.8 |    0.4  |                |
| 66 | F          |           66 | PRO            | nonpolar   | loop      | -         |       0.433824  |  -78   |  176.3 |    0.4  |                |
| 67 | F          |           67 | THR            | polar      | loop      | -         |       0.901408  | -107.5 |  -22.5 |    0.4  |                |
| 68 | F          |           68 | ARG            | positive   | loop      | -         |       0.516129  | -178.2 |  360   |    0.4  |                |

In [64]:
from Bio.PDB import PDBParser
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from Bio.Data import IUPACData
from Bio.SeqUtils import ProtParamData

# 3-letter to 1-letter code map
three_to_one = IUPACData.protein_letters_3to1

# Load PDB
pdb_file = "./../target/1yi5.pdb"
parser = PDBParser()
structure = parser.get_structure("1yi5", pdb_file)

# Extract sequence and map residues
residue_list = []
sequence = ""

for chain in structure[0]:  # First model
    if chain.id == 'F':
        for residue in chain:
            resname = residue.get_resname().strip()
            try:
                one_letter = three_to_one[resname.capitalize()]
                sequence += one_letter
                residue_list.append(residue)
            except KeyError:
                continue  # Skip non-standard residues

print("Sequence:", sequence)
print("Length:", len(sequence))

# Hydropathy analysis with window
window = 9
pa = ProteinAnalysis(sequence)
hydropathy_values = pa.protein_scale(param_dict=ProtParamData.kd, window=window)

# Align hydropathy values to the residue list
offset = window // 2  # Values are centered in the window

for i, residue in enumerate(residue_list):
    resname = residue.get_resname()
    resnum = residue.get_id()[1]
    if offset <= i < len(residue_list) - offset:
        hydro = hydropathy_values[i - offset]
        print(f"Residue {resname} {resnum:>3}: Hydropathy = {hydro:.2f}")
    else:
        print(f"Residue {resname} {resnum:>3}: Hydropathy = NA")


Sequence: IRCFITPDITSKDCPNGHVCYTKTWCDAFCSIRGKRVDLGCAATCPTVKTGVDIQCCSTDNCNPFPTR
Length: 68
Residue ILE   1: Hydropathy = NA
Residue ARG   2: Hydropathy = NA
Residue CYS   3: Hydropathy = NA
Residue PHE   4: Hydropathy = NA
Residue ILE   5: Hydropathy = 0.94
Residue THR   6: Hydropathy = 0.37
Residue PRO   7: Hydropathy = 0.78
Residue ASP   8: Hydropathy = 0.07
Residue ILE   9: Hydropathy = -0.63
Residue THR  10: Hydropathy = -0.86
Residue SER  11: Hydropathy = -0.96
Residue LYS  12: Hydropathy = -1.17
Residue ASP  13: Hydropathy = -0.82
Residue CYS  14: Hydropathy = -1.68
Residue PRO  15: Hydropathy = -1.13
Residue ASN  16: Hydropathy = -0.77
Residue GLY  17: Hydropathy = -0.48
Residue HIS  18: Hydropathy = -0.17
Residue VAL  19: Hydropathy = -0.88
Residue CYS  20: Hydropathy = -0.78
Residue TYR  21: Hydropathy = -0.49
Residue THR  22: Hydropathy = -0.17
Residue LYS  23: Hydropathy = -0.20
Residue THR  24: Hydropathy = -0.47
Residue TRP  25: Hydropathy = -0.43
Residue CYS  26: Hydropath

In [65]:
import pandas as pd
from Bio.PDB import PDBParser
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from Bio.Data import IUPACData
from Bio.SeqUtils import ProtParamData

def calculate_hydropathy_df(pdb_file, chain_id='F', window=9):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("structure", pdb_file)
    
    three_to_one = IUPACData.protein_letters_3to1

    # Extract sequence and residues
    sequence = ""
    residue_info = []

    for chain in structure[0]:  # first model
        if chain.id == chain_id:
            for residue in chain:
                resname = residue.get_resname().strip()
                try:
                    one_letter = three_to_one[resname.capitalize()]
                    sequence += one_letter
                    residue_info.append({
                        'residue_name': resname,
                        'residue_id': residue.get_id()[1],
                        'chain_id': chain.id,
                        'one_letter': one_letter
                    })
                except KeyError:
                    continue  # skip non-standard residues

    # Calculate hydropathy
    analysed_seq = ProteinAnalysis(sequence)
    hydro_values = analysed_seq.protein_scale(param_dict=ProtParamData.kd, window=window)
    offset = window // 2

    # Add hydropathy to residue info
    for i in range(len(residue_info)):
        if offset <= i < len(residue_info) - offset:
            residue_info[i]['hydropathy'] = round(hydro_values[i - offset], 3)
        else:
            residue_info[i]['hydropathy'] = None

    # Add index
    for i, r in enumerate(residue_info):
        r['index'] = i + 1

    return pd.DataFrame(residue_info)

#  Run this
df_hydro = calculate_hydropathy_df("./../target/1yi5.pdb", chain_id='F', window=9)
print(df_hydro.head())

  residue_name  residue_id chain_id one_letter  hydropathy  index
0          ILE           1        F          I         NaN      1
1          ARG           2        F          R         NaN      2
2          CYS           3        F          C         NaN      3
3          PHE           4        F          F         NaN      4
4          ILE           5        F          I       0.944      5


In [66]:
# Ensure consistent residue name format
df_hydro = df_hydro[['chain_id', 'residue_id', 'residue_name', 'hydropathy']]
df_hydro['residue_name'] = df_hydro['residue_name'].str.upper()

# Merge only hydropathy into df_target
df_merged = df_target.merge(
    df_hydro,
    on=['chain_id', 'residue_id', 'residue_name'],
    how='left'  # keep all rows from df_target
)

# Get list of columns
cols = df_merged.columns.tolist()
# Find the index of 'accessibility' column
access_idx = cols.index('accessibility')
# Remove 'hydropathy' from its current location
cols.remove('hydropathy')
# Insert 'hydropathy' right after 'accessibility'
cols.insert(access_idx + 1, 'hydropathy')
# Reorder dataframe
df_merged = df_merged[cols]

# df_merged = df_merged.sort_values(by=['accessibility', 'hydropathy'], ascending=[False, False]).reset_index(drop=True)

# Display result
from IPython.display import display, Markdown
display(Markdown(df_merged.to_markdown(index=False)))

| chain_id   |   residue_id | residue_name   | property   | ss_type   | ss_code   |   accessibility |   hydropathy |    phi |    psi |   plddt | is_interface   |
|:-----------|-------------:|:---------------|:-----------|:----------|:----------|----------------:|-------------:|-------:|-------:|--------:|:---------------|
| F          |            1 | ILE            | nonpolar   | loop      | -         |       0.16568   |      nan     |  360   |  135.5 |    0.39 |                |
| F          |            2 | ARG            | positive   | sheet     | E         |       0.41129   |      nan     |  -77.2 |  117.5 |    0.39 |                |
| F          |            3 | CYS            | polar      | sheet     | E         |       0         |      nan     | -126.1 |  150.2 |    0.39 |                |
| F          |            4 | PHE            | nonpolar   | sheet     | E         |       0.258883  |      nan     |  -69.4 |  112.2 |    0.39 |                |
| F          |            5 | ILE            | nonpolar   | loop      | -         |       0.319527  |        0.944 |  -95.9 |  150.5 |    0.39 |                |
| F          |            6 | THR            | polar      | loop      | S         |       0.140845  |        0.367 | -112.8 |  147.8 |    0.39 |                |
| F          |            7 | PRO            | nonpolar   | loop      | S         |       0.294118  |        0.778 |  -93   |   23.6 |    0.39 |                |
| F          |            8 | ASP            | negative   | loop      | S         |       0.466258  |        0.067 |  -89.2 |  152.2 |    0.4  |                |
| F          |            9 | ILE            | nonpolar   | loop      | S         |       0.650888  |       -0.633 |  -79.6 |  -14.2 |    0.4  |                |
| F          |           10 | THR            | polar      | loop      | S         |       0.43662   |       -0.856 | -101.8 |  165.2 |    0.39 |                |
| F          |           11 | SER            | polar      | sheet     | E         |       0.292308  |       -0.956 |  -94   |  143.7 |    0.39 |                |
| F          |           12 | LYS            | positive   | sheet     | E         |       0.458537  |       -1.167 | -140.3 |  161.5 |    0.4  |                |
| F          |           13 | ASP            | negative   | sheet     | E         |       0.656442  |       -0.822 |  -90.2 |  102.3 |    0.4  |                |
| F          |           14 | CYS            | polar      | loop      | -         |       0.118519  |       -1.678 | -106.6 |  100   |    0.4  |                |
| F          |           15 | PRO            | nonpolar   | loop      | S         |       0.794118  |       -1.133 |  -59.1 |  117.1 |    0.4  |                |
| F          |           16 | ASN            | polar      | loop      | S         |       0.675159  |       -0.767 |    8.7 |   70.7 |    0.39 |                |
| F          |           17 | GLY            | nonpolar   | loop      | -         |       0.142857  |       -0.478 |  -83   |  112.5 |    0.4  |                |
| F          |           18 | HIS            | positive   | loop      | -         |       0.63587   |       -0.167 |  -70.6 |   -3.3 |    0.4  |                |
| F          |           19 | VAL            | nonpolar   | sheet     | E         |       0.126761  |       -0.878 | -137   |  141.5 |    0.39 |                |
| F          |           20 | CYS            | polar      | sheet     | E         |       0         |       -0.778 |  -95.5 |  130.2 |    0.39 |                |
| F          |           21 | TYR            | polar      | sheet     | E         |       0.193694  |       -0.489 | -124.1 |  153.5 |    0.39 |                |
| F          |           22 | THR            | polar      | sheet     | E         |       0.140845  |       -0.167 | -135.3 |   84.4 |    0.39 |                |
| F          |           23 | LYS            | positive   | sheet     | E         |       0.321951  |       -0.2   |  -69.2 |  138.9 |    0.4  |                |
| F          |           24 | THR            | polar      | sheet     | E         |       0.190141  |       -0.467 | -149.2 |  124.5 |    0.4  |                |
| F          |           25 | TRP            | nonpolar   | sheet     | E         |       0.14978   |       -0.433 | -147.4 |  149.1 |    0.39 |                |
| F          |           26 | CYS            | polar      | loop      | -         |       0.318519  |       -0.011 |  -93   |  119.6 |    0.4  |                |
| F          |           27 | ASP            | negative   | loop      | -         |       0.0858896 |       -0.022 | -101.9 | -149.4 |    0.39 |                |
| F          |           28 | ALA            | nonpolar   | loop      | S         |       0.0283019 |        0.911 |  -85.5 |  -40.6 |    0.39 |                |
| F          |           29 | PHE            | nonpolar   | loop      | T         |       0.0507614 |        0.489 |  -69.6 |   23.4 |    0.39 |                |
| F          |           30 | CYS            | polar      | helix     | H         |       0.0444444 |        0.544 | -107.2 |  -36.9 |    0.39 |                |
| F          |           31 | SER            | polar      | helix     | H         |       0.415385  |       -0.167 |  -65.1 |  -25.5 |    0.39 |                |
| F          |           32 | ILE            | nonpolar   | helix     | H         |       0.224852  |       -0.278 |  -84.1 |  -73   |    0.39 |                |
| F          |           33 | ARG            | positive   | helix     | H         |       0.0483871 |       -0.011 |  -95.1 |   -2.2 |    0.39 |                |
| F          |           34 | GLY            | nonpolar   | loop      | -         |       0.214286  |       -0.711 |  114.6 | -177.9 |    0.39 |                |
| F          |           35 | LYS            | positive   | loop      | -         |       0.326829  |       -0.567 |  -58.7 |  146.8 |    0.39 |                |
| F          |           36 | ARG            | positive   | sheet     | E         |       0.314516  |       -0.522 |  -73.8 |  146.9 |    0.39 |                |
| F          |           37 | VAL            | nonpolar   | sheet     | E         |       0.0211268 |       -0.744 | -134.8 |  121.2 |    0.39 |                |
| F          |           38 | ASP            | negative   | sheet     | E         |       0.319018  |       -0.044 | -106   |  114.1 |    0.39 |                |
| F          |           39 | LEU            | nonpolar   | sheet     | E         |       0.054878  |        0.2   |  -90   |  166.5 |    0.39 |                |
| F          |           40 | GLY            | nonpolar   | sheet     | E         |       0.047619  |        0.556 |  162.2 | -171.9 |    0.39 |                |
| F          |           41 | CYS            | polar      | sheet     | E         |       0.237037  |        1.333 | -122.4 |  162.8 |    0.4  |                |
| F          |           42 | ALA            | nonpolar   | sheet     | E         |       0.254717  |        0.689 | -163.5 |  168.6 |    0.39 |                |
| F          |           43 | ALA            | nonpolar   | loop      | S         |       0.603774  |        1     |  -63.4 |  -49.8 |    0.39 |                |
| F          |           44 | THR            | polar      | loop      | S         |       0.753521  |        1.044 | -138.3 |  169.9 |    0.4  |                |
| F          |           45 | CYS            | polar      | loop      | P         |       0.503704  |        0.656 |  -65.3 |  135.3 |    0.39 |                |
| F          |           46 | PRO            | nonpolar   | loop      | P         |       0.330882  |        0.3   |  -74.9 |  168.4 |    0.39 |                |
| F          |           47 | THR            | polar      | loop      | P         |       0.978873  |        0.056 |  -69   |  154.8 |    0.39 |                |
| F          |           48 | VAL            | nonpolar   | loop      | -         |       0.619718  |        0.322 | -112.8 |  115.6 |    0.4  |                |
| F          |           49 | LYS            | positive   | loop      | -         |       0.756098  |        0.011 |  -72.8 |  153.3 |    0.4  |                |
| F          |           50 | THR            | polar      | loop      | S         |       1         |        0.233 |  -75   |  122.2 |    0.4  |                |
| F          |           51 | GLY            | nonpolar   | loop      | S         |       0.619048  |        0.022 |   98.2 |  -37   |    0.39 |                |
| F          |           52 | VAL            | nonpolar   | loop      | -         |       0.197183  |        0.378 |  -51.9 |  143.8 |    0.39 |                |
| F          |           53 | ASP            | negative   | loop      | B         |       0.521472  |        0.189 | -123.2 |  126.6 |    0.4  |                |
| F          |           54 | ILE            | nonpolar   | loop      | -         |       0.284024  |        0.533 | -118   |  161.8 |    0.4  |                |
| F          |           55 | GLN            | polar      | loop      | -         |       0.717172  |        0.533 | -157.3 |  103.9 |    0.39 |                |
| F          |           56 | CYS            | polar      | sheet     | E         |       0.325926  |        0.189 |  -93.7 |  162.2 |    0.39 |                |
| F          |           57 | CYS            | polar      | sheet     | E         |       0.244444  |       -0.667 | -173.1 |  177.3 |    0.39 |                |
| F          |           58 | SER            | polar      | loop      | -         |       0.715385  |        0     | -141.9 |   12.9 |    0.39 |                |
| F          |           59 | THR            | polar      | loop      | S         |       0.683099  |       -0.889 |  -88.4 |  133.8 |    0.39 |                |
| F          |           60 | ASP            | negative   | loop      | S         |       0.662577  |       -0.678 |  -50   |  107.9 |    0.4  |                |
| F          |           61 | ASN            | polar      | loop      | S         |       0.33121   |       -0.644 |   63.7 |   81   |    0.39 |                |
| F          |           62 | CYS            | polar      | loop      | -         |       0.281481  |       -1.1   | -125.6 |    6.3 |    0.39 |                |
| F          |           63 | ASN            | polar      | loop      | -         |       0.0382166 |       -1.089 | -126.8 |   31   |    0.39 |                |
| F          |           64 | PRO            | nonpolar   | loop      | -         |       0.492647  |       -1.511 |  -64.6 | -177.9 |    0.39 |                |
| F          |           65 | PHE            | nonpolar   | loop      | -         |       0.365482  |      nan     |  -73   |  133.8 |    0.4  |                |
| F          |           66 | PRO            | nonpolar   | loop      | -         |       0.433824  |      nan     |  -78   |  176.3 |    0.4  |                |
| F          |           67 | THR            | polar      | loop      | -         |       0.901408  |      nan     | -107.5 |  -22.5 |    0.4  |                |
| F          |           68 | ARG            | positive   | loop      | -         |       0.516129  |      nan     | -178.2 |  360   |    0.4  |                |

In [67]:
df_filtered = df_merged[(df_merged['accessibility'] > 0.7)]
display(Markdown(df_filtered.to_markdown(index=False)))

| chain_id   |   residue_id | residue_name   | property   | ss_type   | ss_code   |   accessibility |   hydropathy |    phi |   psi |   plddt | is_interface   |
|:-----------|-------------:|:---------------|:-----------|:----------|:----------|----------------:|-------------:|-------:|------:|--------:|:---------------|
| F          |           15 | PRO            | nonpolar   | loop      | S         |        0.794118 |       -1.133 |  -59.1 | 117.1 |    0.4  |                |
| F          |           44 | THR            | polar      | loop      | S         |        0.753521 |        1.044 | -138.3 | 169.9 |    0.4  |                |
| F          |           47 | THR            | polar      | loop      | P         |        0.978873 |        0.056 |  -69   | 154.8 |    0.39 |                |
| F          |           49 | LYS            | positive   | loop      | -         |        0.756098 |        0.011 |  -72.8 | 153.3 |    0.4  |                |
| F          |           50 | THR            | polar      | loop      | S         |        1        |        0.233 |  -75   | 122.2 |    0.4  |                |
| F          |           55 | GLN            | polar      | loop      | -         |        0.717172 |        0.533 | -157.3 | 103.9 |    0.39 |                |
| F          |           58 | SER            | polar      | loop      | -         |        0.715385 |        0     | -141.9 |  12.9 |    0.39 |                |
| F          |           67 | THR            | polar      | loop      | -         |        0.901408 |      nan     | -107.5 | -22.5 |    0.4  |                |

In [68]:
df_filtered = df_merged[(df_merged['accessibility'] > 0.7) & (df_merged['hydropathy'] > 0.4)]
display(Markdown(df_filtered.to_markdown(index=False)))

| chain_id   |   residue_id | residue_name   | property   | ss_type   | ss_code   |   accessibility |   hydropathy |    phi |   psi |   plddt | is_interface   |
|:-----------|-------------:|:---------------|:-----------|:----------|:----------|----------------:|-------------:|-------:|------:|--------:|:---------------|
| F          |           44 | THR            | polar      | loop      | S         |        0.753521 |        1.044 | -138.3 | 169.9 |    0.4  |                |
| F          |           55 | GLN            | polar      | loop      | -         |        0.717172 |        0.533 | -157.3 | 103.9 |    0.39 |                |

In [69]:
df_filtered = df_merged[(df_merged['accessibility'] > 0.6) & (df_merged['hydropathy'] > 0.3)]
display(Markdown(df_filtered.to_markdown(index=False)))

| chain_id   |   residue_id | residue_name   | property   | ss_type   | ss_code   |   accessibility |   hydropathy |    phi |   psi |   plddt | is_interface   |
|:-----------|-------------:|:---------------|:-----------|:----------|:----------|----------------:|-------------:|-------:|------:|--------:|:---------------|
| F          |           43 | ALA            | nonpolar   | loop      | S         |        0.603774 |        1     |  -63.4 | -49.8 |    0.39 |                |
| F          |           44 | THR            | polar      | loop      | S         |        0.753521 |        1.044 | -138.3 | 169.9 |    0.4  |                |
| F          |           48 | VAL            | nonpolar   | loop      | -         |        0.619718 |        0.322 | -112.8 | 115.6 |    0.4  |                |
| F          |           55 | GLN            | polar      | loop      | -         |        0.717172 |        0.533 | -157.3 | 103.9 |    0.39 |                |

In [70]:
df_filtered = df_merged[(df_merged['accessibility'] > 0.5) & (df_merged['hydropathy'] > 0.3)]
display(Markdown(df_filtered.to_markdown(index=False)))

| chain_id   |   residue_id | residue_name   | property   | ss_type   | ss_code   |   accessibility |   hydropathy |    phi |   psi |   plddt | is_interface   |
|:-----------|-------------:|:---------------|:-----------|:----------|:----------|----------------:|-------------:|-------:|------:|--------:|:---------------|
| F          |           43 | ALA            | nonpolar   | loop      | S         |        0.603774 |        1     |  -63.4 | -49.8 |    0.39 |                |
| F          |           44 | THR            | polar      | loop      | S         |        0.753521 |        1.044 | -138.3 | 169.9 |    0.4  |                |
| F          |           45 | CYS            | polar      | loop      | P         |        0.503704 |        0.656 |  -65.3 | 135.3 |    0.39 |                |
| F          |           48 | VAL            | nonpolar   | loop      | -         |        0.619718 |        0.322 | -112.8 | 115.6 |    0.4  |                |
| F          |           55 | GLN            | polar      | loop      | -         |        0.717172 |        0.533 | -157.3 | 103.9 |    0.39 |                |

## Detecting Inter-Chain Contacts
To determine whether a particular chain (e.g., 'F') makes contact with other chains in a PDB structure, you can check for interatomic distances between residues in chain 'F' and residues in all other chains. If any atoms are closer than a distance threshold (commonly ~5 Å), it's considered a contact.

In [71]:
from Bio.PDB import PDBParser, NeighborSearch
from Bio.PDB.Polypeptide import is_aa

def get_chain_contacts(pdb_file, target_chain_id='F', distance_threshold=3.5):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("structure", pdb_file)
    model = structure[0]  # Use the first model
    
    # Collect atoms from chain F and other chains
    target_atoms = []
    other_atoms = []
    
    for chain in model:
        for residue in chain:
            if not is_aa(residue):
                continue
            for atom in residue:
                if chain.id == target_chain_id:
                    target_atoms.append(atom)
                else:
                    other_atoms.append(atom)

    # Search for contacts using NeighborSearch
    ns = NeighborSearch(other_atoms)
    contacts = set()

    for atom in target_atoms:
        neighbors = ns.search(atom.coord, distance_threshold)
        for neighbor in neighbors:
            neighbor_chain = neighbor.get_parent().get_parent().id
            if neighbor_chain != target_chain_id:
                contacts.add(neighbor_chain)

    return list(contacts)

pdb_file = "./../target/1yi5.pdb"
contacting_chains = get_chain_contacts(pdb_file, target_chain_id='F')
print(f"Chain 'F' contacts: {contacting_chains}")

Chain 'F' contacts: ['A', 'B']


### Residue-Level Contact Map
identify residue-residue contacts between chain 'F' and other chains in a PDB file. This means listing which residue in chain 'F' is close to which residue in other chains, based on a distance cutoff (usually ~5.0 Å).

In [3]:
from Bio.PDB import PDBParser, NeighborSearch, is_aa

def get_chain_residue_contacts(pdb_file, target_chain_id='F', distance_threshold=5.0):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("structure", pdb_file)
    model = structure[0]

    target_residues = [res for res in model[target_chain_id] if is_aa(res)]

    # Build list of all atoms (excluding the target chain) and map them to residues
    other_atoms = []
    atom_to_residue = {}
    for chain in model:
        if chain.id == target_chain_id:
            continue
        for residue in chain:
            if not is_aa(residue):
                continue
            for atom in residue:
                other_atoms.append(atom)
                atom_to_residue[atom] = residue

    # Build neighbor search index
    ns = NeighborSearch(other_atoms)

    # Store residue-residue contacts
    contacts = []

    for res_f in target_residues:
        for atom in res_f:
            neighbors = ns.search(atom.coord, distance_threshold)
            for neighbor in neighbors:
                res_other = atom_to_residue[neighbor]
                contact = {
                    'F_residue_name': res_f.get_resname(),
                    'F_residue_id': res_f.get_id()[1],
                    'contact_chain': res_other.get_parent().id,
                    'contact_residue_name': res_other.get_resname(),
                    'contact_residue_id': res_other.get_id()[1]
                }
                contacts.append(contact)

    # Remove duplicates
    contacts = [dict(t) for t in {tuple(d.items()) for d in contacts}]
    return contacts



contacts = get_chain_residue_contacts("./../target/1yi5.pdb", target_chain_id='F', distance_threshold=3.5)

import pandas as pd
df_contacts = pd.DataFrame(contacts)
df_contacts = df_contacts.sort_values(by="F_residue_id")
display(df_contacts)

,F_residue_name,F_residue_id,contact_chain,contact_residue_name,contact_residue_id
8,PRO,7,A,GLU,190
13,PRO,7,A,THR,184
7,ILE,9,A,GLU,190
9,ASP,27,A,TYR,185
6,ALA,28,B,TYR,164
10,PHE,29,A,TYR,185
1,SER,31,B,GLN,55
5,ILE,32,B,MET,114
2,ARG,33,A,TYR,192
11,GLY,34,A,SER,186


In [4]:
f_residue_string = ",".join(df_contacts["F_residue_id"].apply(lambda x: f"F{x}").drop_duplicates())
print(f_residue_string)

F7,F9,F27,F28,F29,F31,F32,F33,F34,F35,F36,F65


In [78]:
contacts = get_chain_residue_contacts("./../out/bindcraft/snake-venom-binder/2506192105/Accepted/1yi5_l105_s971412_mpnn2_model2.pdb", target_chain_id='B', distance_threshold=3)
df_contacts_target = pd.DataFrame(contacts)
display(df_contacts_target)

,F_residue_name,F_residue_id,contact_chain,contact_residue_name,contact_residue_id
0,VAL,93,A,ARG,36
1,TRP,97,A,PRO,46
2,HIS,94,A,ASP,38
3,TYR,86,A,ARG,33
4,GLN,101,A,ALA,42
5,TYR,59,A,ILE,32
6,SER,52,A,ALA,28
7,TYR,86,A,PHE,29
8,SER,52,A,PHE,29
9,ARG,98,A,TYR,21


### Detect Disulfide Bonds in a Chain

In [2]:
from Bio.PDB import PDBParser, is_aa
from Bio.PDB.NeighborSearch import NeighborSearch

def find_disulfide_bonds(pdb_file, chain_id='F', distance_cutoff=2.2):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("struct", pdb_file)
    model = structure[0]
    chain = model[chain_id]
    
    # Get all CYS SG atoms
    sg_atoms = []
    residues = []
    
    for residue in chain:
        if is_aa(residue) and residue.get_resname() == "CYS":
            if "SG" in residue:
                sg_atoms.append(residue["SG"])
                residues.append(residue)

    # Search for SG-SG pairs within cutoff
    disulfides = []
    for i in range(len(sg_atoms)):
        for j in range(i + 1, len(sg_atoms)):
            dist = sg_atoms[i] - sg_atoms[j]
            if dist <= distance_cutoff:
                disulfides.append((residues[i].get_id()[1], residues[j].get_id()[1]))

    return disulfides

pdb_file = "./../target/1yi5.pdb"
disulfide_bonds = find_disulfide_bonds(pdb_file, chain_id='F')
print("Disulfide bonds (residue ID pairs):", disulfide_bonds)

pdb_file = "./../target/7z14.pdb"
disulfide_bonds = find_disulfide_bonds(pdb_file, chain_id='F')
print("Disulfide bonds (residue ID pairs):", disulfide_bonds)


Disulfide bonds (residue ID pairs): [(3, 20), (14, 41), (26, 30), (45, 56), (57, 62)]
Disulfide bonds (residue ID pairs): [(3, 22), (17, 39), (41, 52), (53, 58)]
